## An introduction to CobraPy 

CobraPy is a Python package to help with the construction and simulation of genome-scale metabolic models (GSMs). You can find more info in the [detailed documentation](https://cobrapy.readthedocs.org/en/stable/). Here we will just go through a few examples and show you what you can do with CobraPy. 

This CobraPy installation works on Python 3 which is current and future Python version.
This docker container also provides some additional Python libraries for plotting so you can visualize your results. In particular you can use the following libraries:

- matplotlib for basic plotting
- seaborn for nicer and more complex plots
- numpy for basic algebra
- scipy for scientific calculation (statistics, optimization, etc.)
- pandas to handle complex data sets
- statsmodels for linear modelling
- the IBM cplex solver

Okay, let's get started.

### Reading and inspecting models

CobraPy supports imports for a lot of different formats, the most important probably SBML and the Matlab (.mat) format. This docker image also includes the last version of Recon 2 (v4) in .mat format. So we will read it.

In [ ]:
from cobra.io import load_matlab_model
recon2 = load_matlab_model("/models/Recon2.v04.mat")
recon2

Let's extract some basic information from the model, such as number of reactions and metabolites.

In [ ]:
print( "{} reactions, {} genes, and {} metabolites."
      .format(len(recon2.reactions), len(recon2.genes), len(recon2.metabolites)) )

Okay, now let us find the PFK reaction.

In [ ]:
pfk = recon2.reactions.get_by_id("PFK")
pfk

In [ ]:
pfk.build_reaction_string()

In [ ]:
pfk.gene_reaction_rule

In [ ]:
pfk.metabolites

In [ ]:
print("{} <= v_pfk <= {}".format(pfk.lower_bound, pfk.upper_bound))

In [ ]:
print(pfk.reversibility)
pfk.lower_bound = -1000
print(pfk.reversibility)
pfk.lower_bound = 0

### Flux balance analysis and deletions

Running the standard flux balance analyis, thus
$$
\begin{align}
&\max v_{bm} \\
s.t. \quad&\mathbf{Sv}=0\\ 
& l_i \leq v_i \leq u_i
\end{align}
$$
is pretty simple with

In [ ]:
recon2.optimize()

You can specify solvers via the solver argument. Let's use that to time glpk vs. cplex.

In [ ]:
print("GLPK:")
%timeit recon2.optimize()
print("\nCplex:")
%timeit recon2.optimize(solver="cplex")

Okay. We see that the model can grow. Now let us run all single gene deletions to see which reactions are essential.

In [ ]:
import cobra.flux_analysis as fa
growth_rates, status = fa.single_gene_deletion(recon2, solver="cplex")

In [ ]:
from collections import Counter
Counter(status.values())

Now let's find out which gene deletions are essential. 

In [ ]:
import numpy as np

rates = np.array(list(growth_rates.values()))
genes = np.array(list(growth_rates.keys()))

genes[rates < 1e-6]

This gives very few genes, since many enzymes have isoforms, etc. So single gene deletions always never kill the cell. If you want, you can use the `single_reaction_deletion` function to simulate deltions of whole reactions.

In [ ]:
# fill in the rest here

### Analyzing fluxes 

We already know that the optimal growth rate is obtained via the optimization of the FBA problem

$$
\begin{align}
&\max v_{bm} \\
s.t. \quad&\mathbf{Sv}=0\\ 
& l_i \leq v_i \leq u_i
\end{align}
$$

However, many different flux distributions can yield the optimum. So if we want to obatin fluxes from our FBA we will have to deal with this restriction. As a first step we will calculate the flux variablity. Using the following LP problems:

$$
\begin{align}
&\min/\max v_i \\
s.t. \quad&\mathbf{Sv}=0\\ 
& l_i \leq v_i \leq u_i\\
& v_{bm} = \max v_{bm}
\end{align}
$$

This can be done via the `flux_variability_analysis` in `cobra.flux_analysis`. However, that will take a while even with cplex. So let us consider another option. 

We can assume that most living organisms will choose a flux solution which is "economic", meaning associated with the least cost. If we make that assumption we could use parsimonous FBA which uses the following LP:

$$
\begin{align}
&\min\sum v_i \\
s.t. \quad&\mathbf{Sv}=0\\ 
& l_i \leq v_i \leq u_i\\
& v_{bm} = \max v_{bm}
\end{align}
$$

In [ ]:
sol = fa.optimize_minimal_flux(recon2, solver="cplex")

In [ ]:
fluxes = np.array(sol.x)
len(fluxes[fluxes > 1e-6])

In [ ]:
%matplotlib inline
import seaborn as sns
sns.distplot(np.log(fluxes[fluxes > 1e-6]))

## Gene reaction rules

We can also try to count how many backups there are for each gene. For that we will count the number of "ors" in the gene reaction rules +1.

In [ ]:
import re

rex = re.compile("or")

def count_re(regex, rule): 
    return len(list(regex.finditer(rule)))

In [ ]:
ors = np.array([count_re(rex, r.gene_reaction_rule)+1 for r in recon2.reactions])

print("Redundancy per reaction: {} +- {}.".format(ors.mean(), ors.std()))